In [3]:
import pandas as pd
import numpy as np
import random
import pickle
from random import randint, sample
import os
import re
import time

# load node embedding

In [ ]:
# load target embedding
path = '/media/yuting/TOSHIBA EXT/weibo_new/mtl_n_target_embeddings_p.txt'
with open(path, "r", encoding="utf-8") as f:
    lines = f.readlines()
match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *[-+]?\ *[0-9]+)?')
node2emb = {}
emb = ''
for i in range(len(lines)):
    
    if not lines[i].endswith(']]\n'):
        emb = emb + lines[i]
        continue
    else:
        emb = emb + lines[i]
        l_temp = re.findall(match_number,emb)
        assert len(l_temp)==51
        number = [float(x) for x in l_temp[1:]]
        node2emb[l_temp[0]] =  number
        emb = '' 

In [ ]:
# load source embedding
path = '/media/yuting/TOSHIBA EXT/weibo_new/mtl_n_source_embeddings_p.txt'
with open(path, "r", encoding="utf-8") as f:
    lines2 = f.readlines()
node2emb_source = {}
emb_source = ''
for i in range(len(lines2)):
    if not lines2[i].endswith(']]\n'):
        emb_source = emb_source + lines2[i]
        continue
    else:
        emb_source = emb_source + lines2[i]
        l_temp = re.findall(match_number,emb_source)
        assert len(l_temp)==51
        number = [float(x) for x in l_temp[1:]]
        node2emb_source[l_temp[0]] =  number
        emb_source = ''

# get topic embedding

In [ ]:
# /media/yuting/TOSHIBA EXT/weibo_new/doc
def get_topic_emb(news_index):
    topic_emb_dict = {}
    cnt = 0
    with open('/media/yuting/TOSHIBA EXT/weibo_new/doc', 'rb')as f:
        next(f)
        while True:
            line = f.readline()
            if len(line) == 0:
                break
                
            line = line.split()
            assert len(line) == 202
            if int(line[1]) in news_index.keys(): # you can get rid of this condition if you want all the topic embedding
                new_line = [(int(line[i]),float(line[i+1])) for i in range(2,202,2)]
                new_line.sort(key=lambda x:x[0])
                vec = [new_line[i][1] for i in range(len(new_line))]
                topic_emb_dict[news_index[int(line[1])]] = vec
                cnt += 1
                if cnt%1000==0:
                    print(cnt,line[1])
    topic_emb_mat = [0]*len(news_index)
    for i in topic_emb_dict:
        topic_emb_mat[int(i)] = np.array(topic_emb_dict[i],dtype='float32')
    topic_emb_mat[0] = np.zeros(100, dtype='float32')
    topic_emb_mat = np.array(topic_emb_mat, dtype='float32')
    print(topic_emb_mat.shape)
    return topic_emb_mat

# load repost data

In [ ]:
file_path ='/media/yuting/TOSHIBA EXT/weibo_new/repost_data.txt'
with open(file_path, "r", encoding="gbk") as f:
    lines_repost = f.readlines()
    
postid_reuid_dict = {}
i = 0
while i < (len(lines_repost)):
    A = lines_repost[i].split()[0] # A post_id
    B = int(lines_repost[i].split()[1]) # B num of repost
    for j in range(B):
        postid_reuid_dict.setdefault(A,[]).append(lines_repost[i+1+j].split())
    i = i+1+B

Here again I only used the tweets of news, so I filtered them in a new dict mid_history. You can use postid_reuid_dict directly.

In [ ]:
# filter out useful news
mid_history = {}
for mid in postid_mid_map.keys():
    if int(mid) in news_dict.keys():
        mid_history[mid] = postid_reuid_dict[str(postid_mid_map[mid])]
print(len(postid_reuid_dict))
print(len(mid_history))

In [ ]:
user_history = {}
for k,v in mid_history.items():
#     diffusion_list = mid_history[k]
    for i in range(len(v)):
        if v[i][1] not in user_history:
            user_history.setdefault(v[i][1],[]).append([k,v[i][0]])
        else:
            user_history[v[i][1]].append([k,v[i][0]])

In [ ]:
# set time order
user_history = dict([k,sorted(v,key=lambda x: x[1])] for k,v in user_history.items())
mid_history = dict([k,sorted(v,key=lambda x: x[0])] for k,v in mid_history.items())